In [1]:
from src.countries_hook import CountriesHook

hook = CountriesHook()
hook.parse_countries()
hook.parse_tables()


In [2]:
hook.timezones.query('common_name == "Brazil"')
hook.names

,common_name,official_name
0,American Samoa,American Samoa
0,Peru,Republic of Peru
0,Tonga,Kingdom of Tonga
0,Mayotte,Department of Mayotte
0,Panama,Republic of Panama
...,...,...
0,Bosnia and Herzegovina,Bosnia and Herzegovina
0,Greenland,Greenland
0,Jamaica,Jamaica
0,Malta,Republic of Malta
